In [ ]:
# Class: CSDS-391
# Student: Sanket Makkar
# CaseId: sxm1626
# Homework 4
import random
import queue
import math

class EightPuzzle:
    def __init__(self, initialState=None):
        """
        Initialize the EightPuzzle, optionally specify the state you would like it to be in.
        @param: (String) initialState - optional
        """

        #simple blank-slate puzzleState initially, and we also have a goalState to recall
        self.puzzleState = [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ]
        self.goalState = "0 1 2 3 4 5 6 7 8"

        # Error messages stored here
        self.invalidPuzzleStateMessage = "Error: Invalid puzzle state "
        self.invalidMoveMessage = "Error: Invalid move "
        self.invalidCommandMessage = "Error: Invalid command: "
        self.maxNodesMessage = "Error: maxnodes limit ("

        # This will be how we go from cmd input to commands - look through the table and use the given callback
        self.validCommands = { 
            "setState" : self.setState,
            "printState" : self.printState,
            "move" : self.move,
            "scrambleState" : self.scrambleState,
            "logMessage": self.logMessage,
            "printHeuristicH1": self._printHeuristicH1,
            "printHeuristicH2": self._printHeuristicH2,
            "solve DFS": self.solveDFS,
            "solve BFS": self.solveBFS,
            "solve A*": self.solveAStar,
            "generateTable": self._generateTable,
            "branchingFactorBFS": self._branchingFactorBFS,
            "branchingFactorDFS": self._branchingFactorDFS,
            "branchingFactorAStarH1": self._branchingFactorAStarH1,
            "branchingFactorAStarH2": self._branchingFactorAStarH2
        }
        # To ensure that we always make the right move, when we want to execute a move we look through the table below for where that puts us
        # Invalid moves are marked by [-1, -1]
        self.validMoves = {
            "left" : [-1, -1],
            "right" : [-1, -1],
            "up" : [-1, -1],
            "down" : [-1, -1]
        }

        self.specifiers = {
            "h1": self._heuristicH1,
            "h2": self._heuristicH2
        }

        # Now we set the state if the user specified it on construction. If nothing was specified we start at a random scrambleState.
        if initialState != None:
            self.setState(initialState)
        else:
            self.setState(self.scrambleState(10))

        # And finally, we update the validMoves table (which will basically determine which moves are out of bounds, and where each valid move would take us next)
        self._updateValidMoves()

    def __repr__(self):
        # This just generically updates the way python will represent the puzzle when we try to print the object
        initialStr = self._matrix2String(self.puzzleState) # We represent the puzzle as a 2d matrix, but we convert it to a string via a helper here
        return initialStr.replace("0", " ")

    def printState(self):
        # This will now effectively call repr, get back a string with the zero-tile represented as a " ", and print that
        print(self)

    def setState(self, updateString):
        """
        setState(updateString) -> Set the state of the EightPuzzle.
        @param: (String) updateString --> a string which specifies the new EightPuzzleState (left to right, top to bottom)
        @out: None
        """
        if self._checkValidState(updateString): # doing this validity check basically ensures proper length, argument count, argument type, and presence of a zero tile
            updatedMatrix = self._string2Matrix(updateString) # we transform the string to a matrix via a helper
            self.puzzleState = updatedMatrix # We then simply set puzzleState to this new matrix
            self._updateValidMoves() # now update validMoves table

    def move(self, direction):
        """
        move(direction) -> Move blank tile within EightPuzzle.
        @param: (String) direction --> up, down, left, right (string)
        @out: None
        """
        direction = direction.lower() # case insensitive to arguments

        initialElement = self._findEmptySlot() # find the first zero tile
        
        # if our move was invalid let the user know
        if not self._checkMoveValidity(direction) == True:
            self._invalidMoveMessage()
            return

        # else move our tile appropriately
        self._swapElements(initialElement, self.validMoves.get(direction))

        # now update the move table
        self._updateValidMoves()

    def scrambleState(self, moves):
        """
        scrambleState(moves) -> Starting at a solved state, scramble the EightPuzzle.
        @param: (Int) moves --> number of moves in your scramble starting at solved state
        @out: None
        """
        moveCount = moves
        if (type(moves) is str): moveCount = eval(moves) # str2int conversion happens here
        self.setState(self.goalState) # set us to solved state
        
        # make move number of valid moves
        for i in range(moveCount):
            # choices = a filter of our validMove table to only include valid choices
            choices = self._filterMoves()
            # then randomly pick and execute a move
            choice = random.choice(list(choices.keys()))
            self.move(choice)

    def logMessage(self, message):
        """
        logMessage(message) -> Simple utility meant to help log messages based on input.
        @param: (String) message --> message to log
        @out: None
        """
        lowercaseMessage = message.lower()
        # we append a "\n" to the log message if we have completed a test
        append = "\n" if ("complete" in lowercaseMessage and "test" in lowercaseMessage) else "" 
        print("Log: " + message + append)

    def solveDFS(self, maxNodes=1000, printResults=False):
        """
        solveDFS(maxNodes) -> A method that uses depth-first search to search the puzzleState space for a solution.
        @param: (Optional Integer) maxNodes
        @out: None
        """
        if not isinstance(maxNodes, int):
            maxNodes = eval(maxNodes)

        # A list is effectively a stack in python, with identical runtimes when you use list.append and list.pop
        stack = []

        # We also use a list, that will store states in order, for repeated state checking
        visited = set()
        
        # We represent each node as a tuple, with the current state as the first item, and the parent state to this state as the next, and the depth as the last
        initialState = (self._oneLineMatrix2String(self.puzzleState), [], 0)
        
        # start the DFS stack loop
        stack.append(initialState)
        while stack != []:
            # Pop off the stack for current node and state
            currentNode = stack.pop()
            state, pathList, depth = currentNode
            
            # update the visited as we pop from the stack
            visited.add(state)
            
            # set the state to our current node to begin observing from it's perspective
            self.setState(state)
            
            # check for goal state reached - if so break out of loop
            if self._checkGoalStateReached(state, depth, visited, pathList, printResults): return (depth, visited, pathList)
            if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)

            for move in self._reverseFiltereMoves():
                # move in the directions available and append these positions to the stack
                self.move(move)
                updatedPuzzle = self._oneLineMatrix2String(self.puzzleState)
                if (updatedPuzzle in visited): # if new puzzleState in visited ignore it
                    pass
                else: # else add it to the stack
                    newPathList = pathList + [move] 
                    stack.append((updatedPuzzle, newPathList, depth + 1))
                    if self._checkGoalStateReached(updatedPuzzle, depth + 1, visited, newPathList, printResults): return (depth + 1, visited, newPathList)
                    if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)
                # set the state back to our current node again for the next move
                self.setState(state)

    def solveBFS(self, maxNodes=1000, printResults=True):
        """
        solveBFS(maxNodes) -> A method that uses breadth-first search to search the puzzleState space for a solution.
        @param: (Optional Integer) maxNodes
        @out: None
        """
        if not isinstance(maxNodes, int):
            maxNodes = eval(maxNodes)

        # We will use the queue class for this (python built-in)
        Q = queue.Queue()
        visited = set()
        
        # each node will be a tuple containing one stateString, one path list, and one depth counter
        currentNode = (self._oneLineMatrix2String(self.puzzleState), [], 0)
        Q.put(currentNode)

        while (Q.empty() == False):
            # Pop off in FIFO order
            currentNode = Q.get()
            state, pathList, depth = currentNode
            visited.add(state)

            # set the current state to observe from its perspective
            self.setState(state)

            # checking for if we reached the goalState, or if we have exceeded the node limit
            if self._checkGoalStateReached(state, depth, visited, pathList, printResults): return (depth, visited, pathList)
            if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)

            # for all available moves
            for move in self._filterMoves():
                # identify the move result
                self.move(move)
                updatedPuzzle = self._oneLineMatrix2String(self.puzzleState)
                if updatedPuzzle in visited: # if the move is already in the visited don't worry about it.
                    pass
                else: # if not add the node to our queue
                    newPathList = pathList + [move]
                    Q.put((updatedPuzzle, newPathList, depth + 1)) # pathList should naturally accumilate all data on the path taken to this node
                    visited.add(updatedPuzzle)
                    if self._checkGoalStateReached(updatedPuzzle, depth + 1, visited, newPathList, printResults): return (depth + 1, visited, newPathList)
                    if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)
                        
                self.setState(state)

    def solveAStar(self, heuristic, maxNodes=1000, printResults=True):
        if not isinstance(maxNodes, int):
            maxNodes = eval(maxNodes)
            
        heuristicFunction = self.specifiers.get(heuristic)

        pQ = queue.PriorityQueue()
        startingState = (self._oneLineMatrix2String(self.puzzleState), [], 0)
        visited = set()
        
        pQ.put((0 + heuristicFunction(), startingState))

        while (pQ.empty() == False):
            currentNode = pQ.get()
            state, pathList, depth = currentNode[1]
            visited.add(state)
            
            self.setState(state)

            if self._checkGoalStateReached(state, depth, visited, pathList, printResults): return (depth, visited, pathList)
            if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)

            for move in self._filterMoves():
                self.move(move)
                updatedPuzzle = self._oneLineMatrix2String(self.puzzleState)
                if updatedPuzzle in visited:
                    pass
                else:
                    newPathList = pathList + [move]
                    nextState = (updatedPuzzle, newPathList, depth + 1)
                    pQ.put((depth + 1 + heuristicFunction(), nextState))
                    if self._checkGoalStateReached(updatedPuzzle, depth + 1, visited, newPathList, printResults): return (depth + 1, visited, newPathList)
                    if self._checkExceededMaxNodes(visited, maxNodes): return (-1, visited, maxNodes)
                self.setState(state)
        
    def cmd(self, stringToParse):
        """
        cmd(stringToParse) -> given some string input - usually from a CLI - approrpiately execute the corresponding action on the EightPuzzle
        @param: (String) stringToParse --> some input string calling a command
        @out: None
        """
        # eliminate comments here
        stringWithoutComments = self._removeCommentedPartsOfLine(stringToParse)

        # Now we can easily identify the command - supposedly the first "word" typed into the CLI
        stringWithoutCommentsSplit = stringWithoutComments.split(" ")
        command = stringWithoutCommentsSplit[0]
        command = self._solveCheck(command, stringWithoutCommentsSplit)
        
        # Then identify the remainder of the input (up to the last "word" input into the CLI) as the argument portion
        lastAlphaNumericIdx = self._identifyLastAlphaNumeric(stringWithoutComments)
        specifier = self._checkSpecifier(command, stringWithoutComments[len(command) + 1: lastAlphaNumericIdx])
        arguments = stringWithoutComments[len(command) + len(specifier) + 1: lastAlphaNumericIdx]

        # now we map to the appropriate commmand
        commandDict = self.validCommands
        if command in commandDict.keys(): # if the command is valid...
            callback = commandDict.get(command)
            self._executeCallback(callback, arguments, specifier, stringToParse)
        elif command != "": # if the command is not empty but is invalid...
            self._invalidCommandMessage(stringToParse)
        

    def cmdfile(self, file=None):
        """
        cmdfile(file) -> read in all commands from a specified file and execute each line by line
        @param: (String) file --> some specific path to the relevant file
        @out: None
        """
        # If the file was not given stop
        if file is None:
            self._invalidCommandMessage("File invalid")
            return

        # Open file if valid
        # If invalid then stop right here...
        try:
            with open(file, 'r') as fileContents:
                text = fileContents.read()
        except:
            print("Failed to open file")
            return

        # Identify each line, ignoring empty or new lines
        splitText = text.split("\n")
        linesOfText = [line for line in splitText if line not in {"", "\n"}]

        # And now execute all lines in sequence
        for line in linesOfText:
            print(line)
            self.cmd(line)
            
    def _executeCallback(self, callback, arguments, specifier, context):
        """
        _executeCallback(callback, arguments, context) -> Brief helper method to execute a passed in callback
        @param: (pointer) callback --> A callback to be executed
        @param: (String) arguments --> A set of arguments to pass to the callback
        @param: (String) context --> Some context to inform user if callback fails
        @out: None
        """
        try: 
            if not arguments and not specifier: # if no arguments given, then just execute callback without passing anything in
                callback()
            elif not arguments and specifier: # pass specifier if exists without arguments
                callback(specifier)
            elif not specifier: # pass arguments if exist without specifier
                callback(arguments)
            else: # else just give it the specifier and arguments
                callback(specifier, arguments)
        except: # If the callback failed, simply let the user know that this command was invalid
            self._invalidCommandMessage(context)

    def _removeCommentedPartsOfLine(self, stringToParse):
        """
        _removeCommentedPartsOfLine(stringToParse) -> Quick helper method to remove all commented parts of a string
        @param: (String) stringToParse --> Some input to remove comments from
        @out: (String) String without comments
        """
        # Basic idea is to identify location of "//" or "#" comment indicators, and to remove all text including and after them
        if "//" in stringToParse:
            stringToParse = stringToParse[ : stringToParse.rfind("//")]
        if "#" in stringToParse:
            stringToParse = stringToParse[ : stringToParse.rfind("#")]
        return stringToParse

    def _checkValidState(self, inputString):
        """
        _checkValidState(inputString) -> Helper method to ensure a passed in string for puzzle state (represented as input string) is valid
        @param: (String) inputString -> Some string of numbers and characters formatted as [number][space][number]...
        @out: (Boolean) Whether or not the state was indeed valid
        """
        inputString = inputString.replace("\n", " ")
        
        # Ensure each char is a sequence of [number][space][number]... --> Note that we don't allow for 2+ digit numbers simply because this is an "EightPuzzle" so it is only concievable that someone would provide 8 1 digit numbers
        for charIdx in range(len(inputString)):
            currentChar = inputString[charIdx: charIdx + 1]
            if charIdx % 2 == 1 and not currentChar == " ": # Odd number chars should be a " "
                self._invalidPuzzleStateMessage()
                return False
            elif charIdx % 2 == 0 and not currentChar.isdigit(): # Even number chars should be a number
                self._invalidPuzzleStateMessage()
                return False
    
        # Grab the inputString equivalent of our currentState
        currentString = self._matrix2String(self.puzzleState)
        
        # Then verify the length of each string is identical - in effect verifying that, since the loop above verified the [number][space][number]... formatting, the correct number of [number] arguments was provided.
        if len(currentString) != len(inputString):
            self._invalidPuzzleStateMessage()
            return False
            
        # And finally verify that we have exactly 1 zero-tile in our puzzle
        if inputString.count("0") != 1:
            self._invalidPuzzleStateMessage()
            return False

        return True

    def _solveCheck(self, command, fullInput):
        """
        _solveCheck(command, fullInput) -> a method that augments a command to include the word following "solve" if "solve" itself is provided in the input
        @param: (String) command -> a command to check
        @param: (List) fullInput -> a parsed input to append appropriately to our command
        """
        if command == "solve":
            return command + " " + fullInput[1]
        else:
            return command

    def _checkSpecifier(self, command, postCommand):
        """
        _checkSpecifier(command, postCommand) -> a method that returns a specifier to a solve command, as in solve A* <specifier>.
        @param: (String) command -> a command to check
        @param: (String) postCommand -> all data after the command
        @out: (String) the specifier, or an empty string if no specifier.
        """
        if command == "solve A*":
            for specifier in self.specifiers.keys():
                if specifier in postCommand:
                    return specifier
        return ""
            
    def _matrix2String(self, matrix):
        """
        _matrix2String(matrix) -> Helper method to convert a matrix (2d list) data structure into a string
        @param: (List of Lists) matrix -> A 2d list to convert to a string
        @out: (String) A string representing our matrix
        """
        stringProduced = ""
        for idx, row in enumerate(matrix):
            stringProduced += ' '.join(map(str, row))
            if idx < len(matrix) - 1: 
                stringProduced += "\n"
        return stringProduced
        
    def _string2Matrix(self, string):
        """
        _string2Matrix(string) -> Helper method to convert a string of numbers into a 2d matrix -- assumes input of single digit numbers separated by a space
        @param: (String) string -> some string of numbers and characters formatted as [number][space][number]...
        @out: (List of Lists) a list of lists representing the 2d matrix this string would result in for our puzzleState
        """
        # Remove spaces in our string, and set up a matrixProduced list to return - where elements act as columns,
        # and a columnList to append when it is appropriately populated - where elements act to populate each column
        matrixProduced = []
        noSpaceString = string.replace(" ", "")
        columnList = []
        
        for numIdx in range(len(noSpaceString)):
            if numIdx % len(self.puzzleState[0]) == 0 and numIdx > 0: # append to matrixProduced, and reset column list when columnList matches the size of our current puzzleState columns
                matrixProduced.append(columnList)
                columnList = []
            columnList.append(eval(noSpaceString[numIdx]))
        matrixProduced.append(columnList)

        return matrixProduced

    def _oneLineMatrix2String(self, matrix):
        """
        _oneLineMatrix2String(matrix) -> A method to condense a 2d matrix into a single line string
        @param: (2d List) matrix -> A 2d list to observe
        @out: (String) A string reflecting the 1 line version of the 2d matrix
        """
        return self._matrix2String(matrix).replace("\n", " ")

    def _printSearchOutput(self, nodesToSolution, nodesInFrontier, pathToSolution, printResults):
        """
        _printSearchOutput(string) -> Simple helper that does print formatting for state space search output results
        @param: (Integer) nodesToSolution -> number of nodes it takes to reach the solution
        @param: (List) nodesInFrontier -> a list of nodes created in the search process (the visited)
        @param: (List) pathToSolution -> an explicit [left, right, up, down, ...] path to the solution
        @out: None
        """
        if not printResults:
            return
        print("Nodes created during search: " + str(len(nodesInFrontier)))
        print("Solution length: " +  str(nodesToSolution))
        print("Move sequence:")
        for movement in pathToSolution:
            print(movement)
    
    def _findEmptySlot(self):
        """
        _findEmptySlot(string) -> Helper method to grab the zero tile position
        @param: None
        @out: [rowIdx, columnIdx] Some row, column indexes for the zero-tile within our puzzle
        """
        for row in range(len(self.puzzleState)):
            for column in range(len(self.puzzleState[row])):
                if self.puzzleState[row][column] == 0:
                    return [row, column]
        return None

    def _swapElements(self, idxSet1, idxSet2):
        """
        _swapElements(idxSet1, idxSet2) -> Simple swap helper method between two indexes in our puzzle
        @param: [int, int] idxSet1 --> first idxSet to swap
        @param: [int, int] idxSet2 --> second idxSet to swap
        """
        row1, column1 = idxSet1
        row2, column2 = idxSet2
        elementBucket = self.puzzleState[row1][column1]
        self.puzzleState[row1][column1] = self.puzzleState[row2][column2]
        self.puzzleState[row2][column2] = elementBucket

    def _updateValidMoves(self):
        """
        _updateValidMoves() -> Updates the validMoves list, which either categorizes a move as invalid [-1, -1], or identifies where that move would land the puzzle [row, col]
        @params: None
        @out: None
        """
        rowIdxZero, columnIdxZero = self._findEmptySlot() # grab our zero position
        idxUpperBoundRow, idxUpperBoundColumn = [len(self.puzzleState) - 1, len(self.puzzleState[0]) - 1] # upper bounds for rows, columns in terms of movement is just given by puzzleState dimensions
        
        validMoveLeft, validMoveRight, validMoveUp, validMoveDown = [[-1, -1]] * 4 # prepare to identify moves as invalid if not found as valid
        if columnIdxZero > 0: # column lower-bound check
            validMoveLeft = [rowIdxZero, columnIdxZero - 1]
        if columnIdxZero < idxUpperBoundRow: # column upper-bound check
            validMoveRight = [rowIdxZero, columnIdxZero + 1]
        if rowIdxZero > 0: # row lower-bound check
            validMoveUp = [rowIdxZero - 1, columnIdxZero]
        if rowIdxZero < idxUpperBoundColumn: # row upper-bound check
            validMoveDown = [rowIdxZero + 1, columnIdxZero]

        # set based on resulting position if executed, or specify as invalid
        self.validMoves.update({"left": validMoveLeft})
        self.validMoves.update({"right": validMoveRight})
        self.validMoves.update({"up": validMoveUp})
        self.validMoves.update({"down": validMoveDown})

    def _checkGoalStateReached(self, state, treeDepth, visitedNodes, pathList, printResults):
        if state == self.goalState:
            self._printSearchOutput(treeDepth, visitedNodes, pathList, printResults)
            return True
        return False

    def _checkExceededMaxNodes(self, visitedNodes, maxNodes):
        if len(visitedNodes) > maxNodes:
            self._invalidNodeCountMessage(maxNodes)
            return True
        return False

    def _checkMoveValidity(self, direction):
        """
        _checkMoveValidity(direction) -> Is the move invalid, check based on our encoding of [-1, -1] == invalid
        @param: (String) direction --> up, down, left, right specified direction of movement
        @out: (Boolean) Tells us if our direction is valid or invalid to move towards
        """
        return self.validMoves.get(direction) != [-1, -1]

    def _invalidPuzzleStateMessage(self):
        """
        _invalidPuzzleStateMessage() --> Inform user the desired puzzleState is invalid
        @param: None
        @out: None
        """
        self._invalidMessage(message=self.invalidPuzzleStateMessage)
        
    def _invalidMoveMessage(self):
        """
        _invalidMoveMessage() --> Inform user the desired movement is invalid
        @param: None
        @out: None
        """
        self._invalidMessage(message=self.invalidMoveMessage)
        
    def _invalidCommandMessage(self, currentLine):
        """
        _invalidCommandMessage(currentLine) -> Inform user the CLI input was invalid
        @param: (String) currentLine --> the line that caused this invalidCommand error
        @out: None
        """
        self._invalidMessage(message=self.invalidCommandMessage, problemCausingLine=currentLine)

    def _invalidNodeCountMessage(self, maxNodes):
        """
        _invalidNodeCountMessage(maxNodes) -> Inform the user the nodes generated exceeded the maxNodes limit specified
        @param: (Integer) maxNodes --> the number of maxNodes exceeded
        @out: None
        """
        print(self.maxNodesMessage + str(maxNodes) + ") reached")
    
    def _invalidMessage(self, message, problemCausingLine=""):
        """
        _invalidMessage(message, problemCausingLine) -> General purpose helper method which prints some message to console, and can also inform the user of the line that may have caused the problem
        @param: (String) message -> the message to inform the user of
        @param: (String) problemCausingLine --> an optional argument specifying the particular line that caused this message to appear
        @out: None
        """
        print(message + problemCausingLine)

    def startIndividualCommandLoop(self):
        """
        startIndividualCommandLoop() -> Starts an unending user-promt/puzzle-output interaction. Ends via "Exit()" command. 
        @param: None
        @out: None
        """
        while True:
            userInput = input("Command: ")
            if userInput == "Exit()":
                break
            self.cmd(userInput)

    def _identifyLastAlphaNumeric(self, str):
        """
        identifyLastAlphaNumeric(str) -> identify the last alphanumeric or brace position in a string.
        @param: (String) str --> string to analyze
        @out: (Integer) last alphanumeric or brace position index in a string, or -1 if none found.
        """
        reversedStr = str[::-1]
        for charIdx in range(len(reversedStr)):
            if reversedStr[charIdx].isalnum() or reversedStr[charIdx] in {"(", ")", "[", "]"}:
                return len(str) - charIdx
        return -1

    def _filterMoves(self):
        """
        _filterMoves() -> find available moves that are valid
        @param: None
        @out: (Dict) a dict of valid moves.
        """
        choices = {dir:result for dir, result in self.validMoves.items() if result != [-1, -1]}
        return choices

    def _reverseFiltereMoves(self):
        """
        _reverseFiltereMoves() -> find available moves that are valid in reverse order
        @param: None
        @out: (Dict) a dict of valid moves.
        """
        filteredChoices = self._filterMoves()
        reversedChoices = {dir:result for dir, result in reversed(filteredChoices.items())}
        return reversedChoices

    def _heuristicH1(self):
        """
        _heuristicH1() -> find number of moves from current state to goal state via counting misplaced tiles
        @param: None
        @out: (Integer) moves to goal state as counted by number of misplaced tiles.
        """
        currentState = self._oneLineMatrix2String(self.puzzleState).replace(" ", "")
        goalState = self.goalState.replace(" ", "")
        ctr = 0
        for currDigit, trueDigit in zip(currentState, goalState):
            if currDigit != trueDigit and currDigit != "0":
                ctr += 1
        return ctr

    def _heuristicH2(self):
        """
        _heuristicH2() -> find the sum of the distances of each tile from their locatoin
        @param: None
        @out: The manhattan distance
        """
        ctr = 0
        for row in range(len(self.puzzleState)):
            for column in range(len(self.puzzleState[row])):
                digit = self.puzzleState[row][column]
                if digit != 0:
                    correctRowPos = math.floor(digit / 3)
                    correctColumnPos = digit % 3
                    ctr += abs(row - correctRowPos) + abs(column - correctColumnPos)
        return ctr

    def _printHeuristicH1(self):
        print(self._heuristicH1())
    def _printHeuristicH2(self):
        print(self._heuristicH2())

    def _recursiveSum(self, branchingFactor, depth):
        """
        _recursiveSum(branchingFactor, depth) -> find the sum of a tree's node-count given a branching factor and depth
        @param: (Integer) branchingFactor -> a branching factor to consider
        @param: (Integer) depth -> the depth of the tree
        @out: (Integer) the sum of all nodes in the tree
        """
        sum = 0
        for i in range(depth + 1):
            sum += pow(branchingFactor, i)
        return sum
        
    def _computeBranchingFactor(self, Nodes, depth):
        """
        _computeBranchingFactor(Nodes, depth) -> find the effective branching factor of an algorithm given it's node count and solution depth
        @param: (Integer) Nodes -> This is the total node-count of the tree
        @param: (Integer) depth -> This is the depth of the tree
        @out: (Integer) the effective branching factor as computed by 1 + b*^1 + b*^2 + ... + b*^depth = N + 1
        """
        branchingFactor = 1
        acceptableRange = 0.1
        degreeError = 0.1 * acceptableRange
        
        sumFound = self._recursiveSum(branchingFactor, depth)
        while (not(Nodes - acceptableRange < sumFound and sumFound < Nodes + acceptableRange)):
            degreeError = (sumFound - Nodes)/Nodes * (1/depth)
            branchingFactor -= degreeError
            sumFound = self._recursiveSum(branchingFactor, depth)
            
        return branchingFactor
        
    def _testSearchAlgorithm(self, algorithm, states, specifier=None, noPrint=False):
        """
        _testSearchAlgorithm(algorithm, states, specifier=None) -> print the reuslt of considering any search algorithm specified, and any optional specifier, and compute its branching factor under all starting states specified
        @param: algorithm -> the callback to execute
        @param: (List) states -> the string-list of states to execute
        @param: (optional list) specifier -> an optional specifier to pass to the callback when it is executed (good for A* search)
        @out: None
        """
        resultsArr = []
        for state in states: # for each state in the state table
            self.setState(state) # set state
            # perform the relevant algorithm
            if specifier != None:
                 depth, visited, pathList = algorithm(specifier, printResults=False)
            else:
                 depth, visited, pathList = algorithm(printResults=False)

            # if we have a valid solution...
            if depth != -1:
                results = round(self._computeBranchingFactor(len(visited), depth), 2)
                if not noPrint: print("branchingFactor = " + str(results) + "\n") # print it's output
                resultsArr.append((depth, results))

        return resultsArr
    
    def _branchingFactorBFS(self, state):
        """
        _branchingFactorBFS(state) -> test BFS given a state
        @param: (string) state -> a state to test BFS with
        @out: None
        """
        self._testSearchAlgorithm(self.solveBFS, [state])
        
    def _branchingFactorDFS(self, state):
        """
        _branchingFactorDFS(state) -> test DFS given a state
        @param: (string) state -> a state to test DFS with
        @out: None
        """
        self._testSearchAlgorithm(self.solveDFS, [state])

    def _branchingFactorAStarH1(self, state):
        """
        _branchingFactorAStarH2(state) -> test AStar using heuristic h1 given a state
        @param: (string) state -> a state to test AStar using heuristic h1 with
        @out: None
        """
        self._testSearchAlgorithm(self.solveAStar, [state], "h1")

    def _branchingFactorAStarH2(self, state):
        """
        _branchingFactorAStarH2(state) -> test AStar using heuristic h2 given a state
        @param: (string) state -> a state to test AStar using heuristic h2 with
        @out: None
        """
        self._testSearchAlgorithm(self.solveAStar, [state], "h2")

    def _generateTable(self):
        """
        _generateTable() -> utility method to run a mass test on each of our algorithms (BFS, DFS, A* h1 heuristic, A* h2 heuristic)
        @param: None
        @out: None
        """
        states = []
        i = 4
        while i < 24:
            self.scrambleState(i)
            scrambledState = self._oneLineMatrix2String(self.puzzleState) 
            states.append(scrambledState)
            i += 1
        
        print("-----------------BFS------------------")
        BFSData = self._testSearchAlgorithm(self.solveBFS, states, noPrint=True)
        for depth, results in BFSData:
            print("depth" + str(depth) + " --> b* = " + str(results))
        
        print("------------------------\n")
        print("-----------------DFS------------------")
        DFSData = self._testSearchAlgorithm(self.solveDFS, states, noPrint=True)
        for depth, results in DFSData:
            print("depth" + str(depth) + " --> b* = " + str(results))
        print("------------------------\n")
        print("-----------------A* heuristic h1------------------")
        AStarResults = self._testSearchAlgorithm(self.solveAStar, states, "h1", noPrint=True)
        for depth, results in AStarResults:
            print("depth" + str(depth) + " --> b* = " + str(results))
        print("------------------------\n")
        print("-----------------A* heuristic h2------------------")
        AStarResults = self._testSearchAlgorithm(self.solveAStar, states, "h2", noPrint=True)
        for depth, results in AStarResults:
            print("depth" + str(depth) + " --> b* = " + str(results))
        print("------------------------\n")

# This cell is designed for executing all tests within the testfile (testcmds.txt)
# Note: Expected output can be found in testoutput.txt
puzzle = EightPuzzle("1 4 2 3 0 5 6 7 8")
random.seed(216)
puzzle.cmdfile("testcmds.txt")

In [17]:
puzzle.cmd("generateTable")

-----------------BFS------------------
depth4 --> b* = 1.89
depth1 --> b* = 1
depth4 --> b* = 1.89
depth1 --> b* = 1
depth4 --> b* = 1.64
depth1 --> b* = 1
depth6 --> b* = 2.0
depth1 --> b* = 1.91
depth10 --> b* = 1.69
depth7 --> b* = 1.74
depth2 --> b* = 1.78
depth3 --> b* = 1.66
depth0 --> b* = 1
depth1 --> b* = 1
depth6 --> b* = 1.78
depth5 --> b* = 1.82
depth6 --> b* = 1.91
depth9 --> b* = 1.76
depth4 --> b* = 1.67
depth7 --> b* = 1.76
------------------------

-----------------DFS------------------
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
Error: maxnodes limit (1000) reached
depth376 --> b* = 1.0
depth1 --> b* = 0.0
depth26 --> b* = 1.0
depth1 --> b* = 0.0
depth4 --> b* = 0.9
depth1 --> b* = 0.0
depth1 --> b* = 0.0
depth2 --> b* = 0.64
depth3 --> b* = 0.83
depth0 --> b* = 1
depth1 --> b* = 0.0
depth457 